In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import cv2
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split 
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

In [3]:
f = np.load("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/Features2D.npz")
f_noise = np.load("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/Features_noise.npz")
f_shift = np.load("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/Features_noise.npz")

In [4]:
spec = f['spec'] #(400,1025,1099)
mfcc = f['mfcc'] #(400,20,1099)
mel = f['mel']#(400,128,1099)
y = f['target']

In [5]:
spec_noise = f_noise['spec'] #(400,1025,1099)
mfcc_noise = f_noise['mfcc'] #(400,20,1099)
mel_noise = f_noise['mel']#(400,128,1099)
y_noise = f_noise['target']

In [6]:
spec_shift = f_shift['spec'] #(400,1025,1099)
mfcc_shift = f_shift['mfcc'] #(400,20,1099)
mel_shift = f_shift['mel']#(400,128,1099)
y_shift = f_shift['target']

## 3. Data Preprocessing
### 3.1 Resize the data

In [7]:
mfcc_resized = np.empty((973, 120, 600))
mfcc_noise_resized = np.empty((973, 120, 600))
mfcc_shift_resized = np.empty((973, 120, 600))

for i in range(973):
    mfcc_resized[i] = cv2.resize(mfcc[i], (600, 120))
    mfcc_noise_resized[i] = cv2.resize(mfcc_noise[i], (600, 120))
    mfcc_shift_resized[i] = cv2.resize(mfcc_shift[i], (600, 120))


In [8]:
# Resizing the Spectrogram features
spec_resized = np.empty((973, 300, 300))
spec_noise_resized = np.empty((973, 300, 300))
spec_shift_resized = np.empty((973, 300, 300))

for i in range(973):
    spec_resized[i] = cv2.resize(spec[i], (300, 300))
    spec_noise_resized[i] = cv2.resize(spec_noise[i], (300, 300))
    spec_shift_resized[i] = cv2.resize(spec_shift[i], (300, 300))

In [9]:
# Resizing the Mel features
mel_resized = np.empty((973, 300, 400))
mel_noise_resized = np.empty((973, 300, 400))
mel_shift_resized = np.empty((973, 300, 400))

for i in range(973):
    mel_resized[i] = cv2.resize(mel[i], (400, 300))
    mel_noise_resized[i] = cv2.resize(mel_noise[i], (400, 300))
    mel_shift_resized[i] = cv2.resize(mel_shift[i], (400, 300))

In [10]:
# Assuming that you mistakenly created y with only 400 samples
# Recreate y with the correct number of samples (973 in this case)
# Example: y = np.array([label1, label2, ... , label973])

# Ensure y now has the correct shape
print(f"Shape of y after correction: {y.shape}")


Shape of y after correction: (973,)


### 3.2 Train Test Split

In [11]:
mfcc0_train,mfcc_test,y0_train,y_test = train_test_split(mfcc_resized,y, 
                                                         train_size = 0.7, 
                                                         random_state = 13, 
                                                         stratify= y)
mfcc_train_agumented = np.concatenate((mfcc0_train,mfcc_noise_resized,mfcc_shift_resized))
y1_train_agumented = np.concatenate((y0_train,y_noise,y_shift))

In [12]:
spec0_train,spec_test,y0_train,y_test = train_test_split(spec_resized,y, 
                                                   train_size = 0.7, 
                                                   random_state = 13, 
                                                   stratify= y)
spec_train_agumented = np.concatenate((spec0_train,spec_noise_resized,spec_shift_resized))
y2_train_agumented = np.concatenate((y0_train,y_noise,y_shift))

In [13]:
mel0_train,mel_test,y0_train,y_test = train_test_split(mel_resized,y, 
                                                       train_size = 0.7, 
                                                       random_state = 13, 
                                                       stratify= y)
mel_train_agumented = np.concatenate((mel0_train,mel_noise_resized,mel_shift_resized))
y3_train_agumented = np.concatenate((y0_train,y_noise,y_shift))

In [14]:
np.savez_compressed("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/training models/mfcc_train_test_agumented.npz",
                    mfcc_train = mfcc_train_agumented, 
                    mfcc_test = mfcc_test, 
                    y_train = y1_train_agumented, 
                    y_test = y_test)

In [15]:
np.savez_compressed("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/training models/spec_train_test_agumented.npz",
                    spec_train = spec_train_agumented, 
                    spec_test = spec_test, 
                    y_train = y2_train_agumented, 
                    y_test = y_test)

In [16]:
np.savez_compressed("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/training models/mel_train_test_agumented.npz",
                    mel_train = mel_train_agumented, 
                    mel_test = mel_test, 
                    y_train = y3_train_agumented, 
                    y_test = y_test)

### 3.2 Feature Scaling

In [17]:
spec_scaled = spec_resized / np.amax(spec_resized)
mfcc_scaled = (mfcc_resized - np.mean(mfcc_resized))/np.std(mfcc_resized)
mel_scaled = mel_resized / np.amax(mel_resized)

spec_noise_scaled = spec_noise_resized / np.amax(spec_noise_resized)
mfcc_noise_scaled = (mfcc_noise_resized - np.mean(mfcc_noise_resized))/np.std(mfcc_noise_resized)
mel_noise_scaled = mel_noise_resized / np.amax(mel_noise_resized)

spec_shift_scaled = spec_shift_resized / np.amax(spec_shift_resized)
mfcc_shift_scaled = (mfcc_shift_resized - np.mean(mfcc_shift_resized))/np.std(mfcc_shift_resized)
mel_shift_scaled = mel_shift_resized / np.amax(mel_shift_resized)

### 3.3. Reshape Data

In [18]:
N,row,col = mfcc_scaled.shape
mfcc_reshaped = mfcc_scaled.reshape((N,row,col,1))
N,row,col = mfcc_noise_scaled.shape
mfcc_noise_reshaped = mfcc_noise_scaled.reshape((N,row,col,1))
N,row,col = mfcc_shift_scaled.shape
mfcc_shift_reshaped = mfcc_shift_scaled.reshape((N,row,col,1))

In [19]:
N,row,col = spec_scaled.shape
spec_reshaped = spec_scaled.reshape((N,row,col,1))
N,row,col = spec_noise_scaled.shape
spec_noise_reshaped = spec_noise_scaled.reshape((N,row,col,1))
N,row,col = spec_shift_scaled.shape
spec_shift_reshaped = spec_shift_scaled.reshape((N,row,col,1))

In [20]:
N,row,col = mel_scaled.shape
mel_reshaped = mel_scaled.reshape((N,row,col,1))
N,row,col = mel_noise_scaled.shape
mel_noise_reshaped = mel_noise_scaled.reshape((N,row,col,1))
N,row,col = mel_shift_scaled.shape
mel_shift_reshaped = mel_shift_scaled.reshape((N,row,col,1))

## 4. Classification

In [21]:
# Train the model
def modelBuilder3L(X_train,
                   f1,k1,a1,
                   mr1,mc1,sr1,sc1,
                   f2,k2,a2,
                   mr2,mc2,sr2,sc2,
                   f3,k3,a3,
                   mr3,m3,sr3,sc3,
                   d1,dr1,da1,r1,
                   d2,dr2,da2,r2,
                   num):
    
    '''
    args:
    
    X_train: training data
    f1,k1,a1: num of filters, filter size and activation func of 1st conv2D layer
    mr1,mc1,sr1,sc1: filter size and strides of 1st MaxPooling2D layer
    f2,k2,a2: num of filters, filter size and activation func of 2nd conv2D layer
    mr2,mc2,sr2,sc2: filter size and strides of 2nd MaxPooling2D layer
    f3,k3,a3: num of filters, filter size and activation func of 3rd conv2D layer
    mr3,sc3,sr3,sc3: filter size and strides of 3rd MaxPooling2D layer
    d1,dr1,da1, r1: num of units, dropout, activation func and regularizer para of 1st fully connected layer
    d2,dr2,da2, r2: num of units, dropout, activation func and regularizer para of 2nd fully connected layer
    num: integer for distinguishing different model
    
    return:
    model
    
    '''
    
    N,row,col,Nc = X_train.shape
    
    model = Sequential(name = "CNN2D_"+str(num))
    #L1
    model.add(Conv2D(f1,(k1,k1),activation = a1,input_shape = (row,col,Nc), padding = 'same', name = 'Conv2D_1'))
    model.add(MaxPooling2D((mw1,mh1), strides = (sw1,sh1), padding = 'same', name = 'MaxPooling2D_1'))
    model.add(BatchNormalization(name = "BN1"))
    
    #L2
    model.add(Conv2D(f2,(k2,k2), activation = a2, padding = 'same',name = 'Conv2D_2'))
    model.add(MaxPooling2D((mw2,mh2),strides = (sw2,sh2), padding = 'same', name = 'MaxPooling2D_2'))
    model.add(BatchNormalization(name = "BN2"))

    #L3
    model.add(Conv2D(f3,(k3,k3), activation = a3, padding = 'same',name = 'Conv2D_3'))
    model.add(MaxPooling2D((mw3,mh3), strides = (sw3,sh3), padding = 'same', name = 'MaxPooling2D_3'))
    model.add(BatchNormalization(name = "BN3"))
   
    #Flatten layer
    model.add(Flatten(name = 'Flatten'))
    #FC4
    model.add(Dense(d1, activation = da1, kernel_regularizer = tf.keras.regularizers.L2(r1), name='Dense1'))
    model.add(Dropout(dr1, name = "Dropout1"))
    #FC5
    model.add(Dense(d2, activation = da2, kernel_regularizer = tf.keras.regularizers.L2(r2), name='Dense2'))
    model.add(Dropout(dr2, name = "Dropout2"))
    #Softmax layer
    model.add(Dense(4, activation ='softmax', name='Softmax'))

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer= optimizer, loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

    model.summary()
    
    return model

### 4.1 MFCC

In [22]:
# Load MFCC data set
mfcc_file = np.load("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/training models/mfcc_train_test_agumented.npz")



In [23]:

mfcc_train = mfcc_file['mfcc_train']


In [24]:
y1_train = mfcc_file['y_train']



In [25]:

# validation set
mfcc_train,mfcc_val,y1_train,y1_val = train_test_split(mfcc_train,y1_train, 
                                                       train_size = 0.8,  
                                                       random_state = 13, 
                                                       stratify= y1_train)

MemoryError: Unable to allocate 1.13 GiB for an array with shape (2101, 120, 600) and data type float64

In [ ]:
# Feature Scaling
mfcc_train_scaled = (mfcc_train - np.mean(mfcc_train))/np.std(mfcc_train)
mfcc_val_scaled = (mfcc_val - np.mean(mfcc_val))/np.std(mfcc_val)

In [ ]:
# Reshape the data
N,row,col = mfcc_train_scaled.shape
mfcc_train_reshaped = mfcc_train_scaled.reshape((N,row,col,1))
N,row,col = mfcc_val_scaled.shape
mfcc_val_reshaped = mfcc_val_scaled.reshape((N,row,col,1))

In [ ]:
f1,k1,a1 = 8,3,'relu'
mw1,mh1,sw1,sh1 = 2,3,2,2
f2,k2,a2 = 8,3,'relu'
mw2,mh2,sw2,sh2 = 2,3,2,2
f3,k3,a3 = 16,3,'relu'
mw3,mh3,sw3,sh3 = 2,3,2,2
d1,dr1,da1,r1 = 64,0.2,'relu',0.2
d2,dr2,da2,r2 = 32,0.2,'relu',0.2
num = 1

model1 = modelBuilder3L(mfcc_train_reshaped,
                        f1,k1,a1,
                        mw1,mh1,sw1,sh1,
                        f2,k2,a2,
                        mw2,mh2,sw2,sh2,
                        f3,k3,a3,
                        mw3,mh3,sw3,sh3,
                        d1,dr1,da1,r1,
                        d2,dr2,da2,r2,
                        num)

In [ ]:
history1 = model1.fit(mfcc_train_reshaped,y1_train,validation_data=(mfcc_val_reshaped,y1_val),batch_size=20,epochs=80)

In [ ]:
# Train and Validation accuracy and loss
f = plt.figure(figsize = (12,4))

plt.subplot(1,2,1)
plt.title('train validation accuracy')
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.legend(['train','validation'])
plt.xlabel('epoch')
plt.ylabel('accuracy')


plt.subplot(1,2,2)
plt.title('train validation loss')
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.legend(['train','validation'])
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
mfcc_test = mfcc_file['mfcc_test']
y_test_mfcc = mfcc_file['y_test']

In [ ]:
# Feature Scaling
mfcc_test_scaled = (mfcc_test - np.mean(mfcc_test))/np.std(mfcc_test)
# Reshape the data
N,row,col = mfcc_test_scaled.shape
mfcc_test_reshaped = mfcc_test_scaled.reshape((N,row,col,1))

In [ ]:
y1_prob = model1.predict(mfcc_test_reshaped)
y1_pred = np.argmax(y1_prob, axis = -1)

In [ ]:
# Accuracy
acc_mfcc = round(accuracy_score(y1_pred,y_test_mfcc),2)

In [ ]:
# Confusion Matrix
cm_mfcc = confusion_matrix(y_test_mfcc,y1_pred)
cm_display_mfcc = ConfusionMatrixDisplay(confusion_matrix = cm_mfcc, display_labels = ['Happy', 'Relaxed', 'Sad'])
print(classification_report(y_test_mfcc, y1_pred, target_names = ['Happy', 'Relaxed', 'Sad']))
cm_display_mfcc.plot()
plt.show()

In [ ]:
model1.save("D:/UM/Project/Mozartify/project/Model/CNN_models/2D_CNN/Conv2D_mfcc_agumented.h5")

### 4.2 Spectrogram 

In [ ]:
# Load Spectrogram data set
spec_file = np.load("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/training models/spec_train_test_agumented.npz")

spec_train = spec_file['spec_train']
y2_train = spec_file['y_train']

spec_train,spec_val,y2_train,y2_val = train_test_split(spec_train,y2_train, 
                                                       train_size = 0.8,
                                                       random_state = 13, 
                                                       stratify= y2_train)


In [ ]:
# Feature Scaling
spec_train_scaled = spec_train / np.amax(spec_train)
spec_val_scaled = spec_val / np.amax(spec_val)

In [ ]:
# Reshape the data
N,row,col = spec_train_scaled.shape
spec_train_reshaped = spec_train_scaled.reshape((N,row,col,1))
N,row,col = spec_val_scaled.shape
spec_val_reshaped = spec_val_scaled.reshape((N,row,col,1))

In [ ]:
f1,k1,a1 = 5,3,'relu'
mw1,mh1,sw1,sh1 = 3,3,2,2
f2,k2,a2 = 8,3,'relu'
mw2,mh2,sw2,sh2 = 3,3,2,2
f3,k3,a3 = 16,3,'relu'
mw3,mh3,sw3,sh3 = 3,3,2,2
d1,dr1,da1,r1 = 64,0.2,'relu',0.2
d2,dr2,da2,r2 = 32,0.2,'relu',0.2
num = 2
model2 = modelBuilder3L(spec_train_reshaped,
                        f1,k1,a1,
                        mw1,mh1,sw1,sh1,
                        f2,k2,a2,
                        mw2,mh2,sw2,sh2,
                        f3,k3,a3,
                        mw3,mh3,sw3,sh3,
                        d1,dr1,da1,r1,
                        d2,dr2,da2,r2,
                        num)

In [ ]:
history2 = model2.fit(spec_train_reshaped,y2_train,validation_data=(spec_val_reshaped,y2_val),batch_size=20,epochs=80)

In [ ]:
# Train and Validation accuracy and loss
f = plt.figure(figsize = (12,4))

plt.subplot(1,2,1)
plt.title('train validation accuracy')
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.legend(['train','validation'])
plt.xlabel('epoch')
plt.ylabel('accuracy')


plt.subplot(1,2,2)
plt.title('train validation loss')
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.legend(['train','validation'])
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
spec_test = spec_file['spec_test']
y_test_spec = spec_file['y_test']

In [ ]:
# Feature Scaling
spec_test_scaled = spec_test / np.amax(spec_test)

# Reshape the data
N,row,col = spec_test_scaled.shape
spec_test_reshaped = spec_test_scaled.reshape((N,row,col,1))

In [ ]:
y2_prob = model2.predict(spec_test_reshaped)
y2_pred = np.argmax(y2_prob, axis = -1)

In [ ]:
# Accuracy
acc_spec = round(accuracy_score(y2_pred,y_test_spec),2)
acc_spec

In [ ]:
# Confusion Matrix
cm_spec = confusion_matrix(y_test_spec,y2_pred)
cm_display_spec = ConfusionMatrixDisplay(confusion_matrix = cm_spec, display_labels = ['Happy', 'Relaxed', 'Sad'])
print(classification_report(y_test_spec, y2_pred, target_names = ['Happy', 'Relaxed', 'Sad']))
cm_display_spec.plot()
plt.show()

In [ ]:
# Save model
model2.save("D:/UM/Project/Mozartify/project/Model/CNN_models/2D_CNN/Conv2D_spec_agumented.h5") 

### 4.3 Mel Spectrogram

In [ ]:
mel_file = np.load("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/training models/mel_train_test_agumented.npz")

mel_train = mel_file['mel_train']
y3_train = mel_file['y_train']
mel_train,mel_val,y3_train,y3_val = train_test_split(mel_train,y3_train, 
                                                     train_size = 0.8, 
                                                     random_state = 13, 
                                                     stratify= y3_train)

In [ ]:
# Feature Scaling
mel_train_scaled = (mel_train - np.mean(mel_train))/np.std(mel_train)
mel_val_scaled = (mel_val - np.mean(mel_val))/np.std(mel_val)

In [ ]:
# Reshape the data
N,row,col = mel_train_scaled.shape
mel_train_reshaped = mel_train_scaled.reshape((N,row,col,1))

N,row,col = mel_val_scaled.shape
mel_val_reshaped = mel_val_scaled.reshape((N,row,col,1))

In [ ]:
f1,k1,a1 = 8,3,'relu'
mw1,mh1,sw1,sh1 = 2,3,2,2
f2,k2,a2 = 8,3,'relu'
mw2,mh2,sw2,sh2 = 2,3,2,2
f3,k3,a3 = 16,3,'relu'
mw3,mh3,sw3,sh3 = 2,3,2,2
d1,dr1,da1,r1 = 64,0.2,'relu',0.2
d2,dr2,da2,r2 = 32,0.2,'relu',0.2
num = 3
model3 = modelBuilder3L(mel_train_reshaped,
                        f1,k1,a1,
                        mw1,mh1,sw1,sh1,
                        f2,k2,a2,
                        mw2,mh2,sw2,sh2,
                        f3,k3,a3,
                        mw3,mh3,sw3,sh3,
                        d1,dr1,da1,r1,
                        d2,dr2,da2,r2,
                        num)

In [ ]:
history3 = model3.fit(mel_train_reshaped,y3_train,validation_data=(mel_val_reshaped,y3_val),batch_size=20,epochs=80)

In [ ]:
# Train and Validation accuracy and loss
f = plt.figure(figsize = (12,4))

plt.subplot(1,2,1)
plt.title('train validation accuracy')
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])
plt.legend(['train','validation'])
plt.xlabel('epoch')
plt.ylabel('accuracy')


plt.subplot(1,2,2)
plt.title('train validation loss')
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.legend(['train','validation'])
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
model3.save("D:/UM/Project/Mozartify/project/Model/CNN_models/2D_CNN/Conv2D_mel_agumented.h5") 

In [ ]:
mel_test = mel_file['mel_test']
y_test_mel = mel_file['y_test']

In [ ]:
mel_test_scaled = (mel_test - np.mean(mel_test))/np.std(mel_test)

# Reshape the data
N,row,col = mel_test_scaled.shape
mel_test_reshaped = mel_test_scaled.reshape((N,row,col,1))

In [ ]:
y3_prob = model3.predict(mel_test_reshaped)
y3_pred = np.argmax(y3_prob, axis = -1)

In [ ]:
# Accuracy
acc_mel = round(accuracy_score(y3_pred,y_test_mel),2)
acc_mel

In [ ]:
# Confusion Matrix
cm_mel = confusion_matrix(y_test_mel,y3_pred)
cm_display_mel = ConfusionMatrixDisplay(confusion_matrix = cm_mel, display_labels = ['Happy', 'Relaxed', 'Sad'])
print(classification_report(y_test_mel, y3_pred, target_names = ['Happy', 'Relaxed', 'Sad']))
cm_display_mel.plot()
plt.show()

## 5. Ensenble 3 Models

In [ ]:
mfcc_file_ense = np.load("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/training models/mfcc_train_test_agumented.npz")
mfcc_ense_train = mfcc_file_ense['mfcc_train']
mfcc_ense_test =mfcc_file_ense['mfcc_test']
y_ense_train = mfcc_file_ense['y_train']
y_ense_test = mfcc_file_ense['y_test']
model_mfcc = load_model("D:/UM/Project/Mozartify/project/Model/CNN_models/2D_CNN/Conv2D_mfcc_agumented.h5")

In [ ]:
spec_file_ense = np.load("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/training models/spec_train_test_agumented.npz")
spec_ense_train = spec_file_ense['spec_train']
spec_ense_test = spec_file_ense['spec_test']
y_ense_train = spec_file_ense['y_train']
y_ense_test = spec_file_ense['y_test']
model_spec = load_model("D:/UM/Project/Mozartify/project/Model/CNN_models/2D_CNN/Conv2D_spec_agumented.h5")

In [ ]:
model_spec = load_model("D:/UM/Project/Mozartify/project/Datasets/Features/Features2D/training models/Conv2D_spec_agumented.h5")
mel_file_ense = np.load("/mel_train_test_agumented.npz")
mel_ense_train = mel_file_ense['mel_train']
mel_ense_test =mel_file_ense['mel_test']
y_ense_train = mel_file_ense['y_train']
y_ense_test = mel_file_ense['y_test']
model_mel = load_model("D:/UM/Project/Mozartify/project/Model/CNN_models/2D_CNN/Conv2D_mel_agumented.h5")

In [ ]:
# Feature Scaling
mfcc_ense_train_scaled = (mfcc_ense_train - np.mean(mfcc_ense_train))/np.std(mfcc_ense_train)
spec_ense_train_scaled = spec_ense_train / np.amax(spec_ense_train)
mel_ense_train_scaled = (mel_ense_train - np.mean(mel_ense_train))/np.std(mel_ense_train)

mfcc_ense_test_scaled = (mfcc_ense_test - np.mean(mfcc_ense_test))/np.std(mfcc_ense_test)
spec_ense_test_scaled = spec_ense_test / np.amax(spec_ense_test)
mel_ense_test_scaled = (mel_ense_test - np.mean(mel_ense_test))/np.std(mfcc_ense_test)


# Reshape the data
N,row,col = mfcc_ense_train_scaled.shape
mfcc_ense_train_reshaped = mfcc_ense_train_scaled.reshape((N,row,col,1))
N,row,col = spec_ense_train_scaled.shape
spec_ense_train_reshaped = spec_ense_train_scaled.reshape((N,row,col,1))
N,row,col = mel_ense_train_scaled.shape
mel_ense_train_reshaped = mel_ense_train_scaled.reshape((N,row,col,1))

N,row,col = mfcc_ense_test_scaled.shape
mfcc_ense_test_reshaped = mfcc_ense_test_scaled.reshape((N,row,col,1))
N,row,col = spec_ense_test_scaled.shape
spec_ense_test_reshaped = spec_ense_test_scaled.reshape((N,row,col,1))
N,row,col = mel_ense_test_scaled.shape
mel_ense_test_reshaped = mel_ense_test_scaled.reshape((N,row,col,1))

In [ ]:
# Train 3 models
y_ense_prob1 = model_mfcc.predict(mfcc_ense_train_reshaped)
y_ense_pred1 = np.argmax(y_ense_prob1, axis= -1)

y_ense_prob2 = model_spec.predict(spec_ense_train_reshaped)
y_ense_pred2 = np.argmax(y_ense_prob2, axis= -1)

y_ense_prob3 = model_mel.predict(mel_ense_train_reshaped)
y_ense_pred3 = np.argmax(y_ense_prob3, axis= -1)

In [ ]:
# Vote for most accuracte prediction
def get_majority(pred) :
    N = len(pred[0]) # num of examples
    vote = [] 
    for i in range(N) :
        elements = [j[i] for j in pred] # 3 Prediction results  using 3 different model
        elements = np.array(elements)
        uniq, freq = np.unique(elements, return_counts= True) # Count the occurence of unqiue result
        vote.append(uniq[np.argmax(freq)])# Choose the prediction with most occurence
    
    vote = np.array(vote)
    return vote

In [ ]:
# 3 different predictions
train_pred = [y_ense_pred1, y_ense_pred2, y_ense_pred3]

In [ ]:
y_train_pred = get_majority(train_pred)

In [ ]:
# Model Evaluation
train_corr = len(y_train_pred) - np.count_nonzero(y_train_pred - y_ense_train)
train_acc = np.round((train_corr/ len(y_train_pred)),4)
train_acc

In [ ]:
# Test ensembled models
y_test_prob1 = model_spec.predict(spec_ense_test_reshaped)
y_test_pred1 = np.argmax(y_test_prob1, axis= -1)

y_test_prob2 = model_mfcc.predict(mfcc_ense_test_reshaped)
y_test_pred2 = np.argmax(y_test_prob2, axis= -1)

y_test_prob3 = model_mel.predict(mel_ense_test_reshaped)
y_test_pred3 = np.argmax(y_test_prob3, axis= -1)

In [ ]:
test_pred = [y_test_pred1, y_test_pred2, y_test_pred3]

In [ ]:
y_test_pred = get_majority(test_pred)

In [ ]:
# Model Evaluation
test_corr = len(y_test_pred) - np.count_nonzero(y_test_pred - y_ense_test)
test_acc = np.round((test_corr/ len(y_test_pred)),4)
test_acc

In [ ]:
# Confusion Matrix
cm_ense = confusion_matrix(y_ense_test,y_test_pred)
cm_display_ense = ConfusionMatrixDisplay(confusion_matrix = cm_ense, display_labels = ['Happy', 'Relaxed', 'Sad'])
print(classification_report(y_ense_test, y_test_pred, target_names = ['Happy', 'Relaxed', 'Sad']))
cm_display_ense.plot()
plt.show()